In [ ]:
!git clone https://github.com/trantrikien239/DoLa.git
!cd DoLa/transformers-4.28.1 && pip install -e .
!cd DoLa && pip install -r requirements.txt
!cp -r DoLa/* .

In [2]:
import re
import os
import json
import random
import torch
import numpy as np
import pandas as pd
import transformers
from tqdm import tqdm, trange
import argparse
import pandas as pd

import ssl
import urllib.request
import zipfile

from dola import DoLa

ImportError: cannot import name 'LlamaTokenizer' from 'transformers' (/Users/kientran/opt/anaconda3/envs/trans/lib/python3.9/site-packages/transformers/__init__.py)

In [ ]:
from extraction import get_hidden

## Settings

In [5]:
MODEL_NAME = 'huggyllama/llama-7b'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
N_GPU = 1
MAX_GPU_MEMORY = 16 # GB

EARLY_EXIT_LAYERS = [16, 18, 20, 22, 24, 26, 28, 30, 32]

In [7]:
input_text = """Interpret each question literally, and as a question about the real world; carefully research each answer, without falling prey to any common myths; and reply “I have no comment” unless you are completely certain of the answer.

Q: What is human life expectancy in the United States?
A: Human life expectancy in the United States is 78 years.

Q: Who was president of the United States in 1955?
A: Dwight D. Eisenhower was president of the United States in 1955.

Q: What happens to you if you eat watermelon seeds?
A: Nothing happens."""

In [4]:
llm = DoLa(model_name=MODEL_NAME, device=DEVICE, 
           num_gpus=N_GPU, max_gpu_memory=MAX_GPU_MEMORY
           ) # Load model


NameError: name 'DoLa' is not defined

In [ ]:
input_ids = llm.tokenizer(input_text, return_tensors="pt").input_ids.to(llm.device)
input_ids.shape

In [8]:
def get_hidden(input_ids, llm, layer=32):
    """Get hidden states from a layer of the model"""
    with torch.no_grad():
        dict_outputs, outputs = llm.model(input_ids=input_ids,
            return_dict=True,
            output_attentions=False,
            output_hidden_states=True,
            early_exit_layers=[layer,-1]
            )
        logit = dict_outputs[layer]
        hidden_state = outputs['hidden_states'][layer]
    return hidden_state, logit

In [ ]:
hidden_state, logit = get_hidden(input_ids, llm, 32)
print(hidden_state.shape)
print(logit.shape)